# Handling HP

As our players defeat monsters, or heaven forbid even lose!) they are going to lose HP and we need a mechanism that handles that for us. As with many games, we'll create a mechanic where HP regens over time. While you might initialy reach for some sort of timer to regularly update values, we are going to think like a programer and handle things a bit differently.

Currently we have no checks on a player's HP other than if they win or lose combat. We need to add two checks- we want to set a minimum value of zero and we want to prevent a player from starting a new encounter with zero health.

Updating the player `take_damage()` method is pretty straightforward:

In [ ]:
def take_damage(self, damage) -> bool:
    self.current_hp -= damage
    if self.current_hp < 0:
        self.current_hp = 0
    return self.is_alive()

Then in the encounter class we need to add our checks. There are two places to add checks. First is in the `create_encounter()` method, and the second is in the `player_action()` method where we create an encounter if there isn't one already.

In [ ]:
def create_encounter(self, player_id):
    encounter = self.encounters.get(player_id)
    print(encounter)
    if encounter:
        return f"""Current Status:
        Player HP: {self.players[player_id].current_hp}
        Mob HP: {encounter.mob.current_hp}"""
    else:
        player = self.players[player_id]
        if player.is_alive():
            self.encounters[player_id] = Encounter(player)
            return "Encounter created!"
        else:
            return "Can't start a fight with no health!"

def player_action(self, player_id):
    encounter = self.encounters.get(player_id)
    if encounter is None:
        new_encounter = self.createEncounter(player_id)
        encounter = self.encounters[player_id]
        if encounter is None:
            return new_encounter
    #rest of this method continues

## Creating regen

So this is where we need to think like a coder. We could set a timer to regularly update player health, but we don't need that data all the time. We only care when the player is interacting with the bot. So we'll use a different strategy. We'll keep track of the last time their health changed. Then when they access that value we'll calculate what the value would be if we had used a timer.

There's a couple things to add here. A `last_hp_update` property that holds the value. we are then also going to do a little magic to create a "getter" this is a special function that's called whenever we try to access a value. We'll use that to update the HP whenever we access it.

In [ ]:
class Player:
    def __init__(self, id, xp=0, attack=0, hp=0):
        self.id = id
        self.xp = xp
        self.attack = attack
        self.max_hp = hp
        self._current_hp = hp
        self.last_hp_update = datetime.now()

    @property
    def current_hp(self):
        now = datetime.now()
        if self._current_hp < self.max_hp:
            time_passed = now - self.last_hp_update
            hp_change = (time_passed.total_seconds() - 30) * .1
            if hp_change + self._current_hp > self.max_hp:
                self._current_hp = self.max_hp
            else:
                self._current_hp += hp_change

        self.last_hp_update = now
        return self._current_hp

    def do_attack(self) -> int:
        #rest of the class continues